In [1]:
!pip install nltk
!pip install pandas
!pip install sklearn
!pip install gensim
"""
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import sklearn
"""

"\nimport nltk\nnltk.download('stopwords')\nnltk.download('punkt')\nnltk.download('wordnet')\nimport sklearn\n"

In [2]:
import pandas as pd
traindf = pd.read_csv('labeledTrainData.tsv', delimiter="\t")
traindf = traindf.drop(['id'], axis=1)
traindf.head()

,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


In [3]:
import re
import string
# import the inflect library 
import inflect 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer 

stemmer = PorterStemmer() 
p = inflect.engine() 
stop_words = set(stopwords.words("english")) 
# remove stopwords function 
def remove_stopwords(text): 
    word_tokens = word_tokenize(text) 
    filtered_text = [word for word in word_tokens if word not in stop_words] 
    filtered_text = " ".join(filtered_text)
    return filtered_text 

def text_lowercase(text): 
    return text.lower() 
  
# convert number into words 
def convert_number(text): 
    # split string into list of words 
    temp_str = text.split() 
    # initialise empty list 
    new_string = [] 
  
    for word in temp_str: 
        # if word is a digit, convert the digit 
        # to numbers and append into the new_string list 
        if word.isdigit(): 
            temp = p.number_to_words(word) 
            new_string.append(temp) 
  
        # append the word as it is 
        else: 
            new_string.append(word) 
  
    # join the words of new_string to form a string 
    temp_str = ' '.join(new_string) 
    return temp_str

# remove punctuation 
def remove_punctuation(text): 
    translator = str.maketrans('', '', string.punctuation) 
    return text.translate(translator)

# remove whitespace from text 
def remove_whitespace(text): 
    return  " ".join(text.split()) 

lemmatizer = WordNetLemmatizer() 
# lemmatize string 
def lemmatize_word(text): 
    word_tokens = word_tokenize(text) 
    # provide context i.e. part-of-speech 
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in word_tokens] 
    #lemmas = " ".join(lemmas)
    return lemmas 
# stem words in the list of tokenised words 
def stem_words(text): 
    word_tokens = word_tokenize(text) 
    stems = [stemmer.stem(word) for word in word_tokens] 
    stems = " ".join(stems)
    return stems 

def preprocess_text(text):
    text = text_lowercase(text)
    text = convert_number(text)
    text = remove_punctuation(text)
    text = remove_whitespace(text)
    text = remove_stopwords(text)
    #text = stem_words(text)
    text = lemmatize_word(text)
    return text

traindf['processed_review'] = traindf.review.apply(lambda x: preprocess_text(x))  
print("Number of reviews: ",len(traindf))
traindf.head(3)

Number of reviews:  25000


,sentiment,review,processed_review
0,1,With all this stuff going down at the moment w...,"[stuff, go, moment, mj, ive, start, listen, mu..."
1,1,"\The Classic War of the Worlds\"" by Timothy Hi...","[classic, war, worlds, timothy, hines, enterta..."
2,0,The film starts with a manager (Nicholas Bell)...,"[film, start, manager, nicholas, bell, give, w..."


In [4]:
#BAG OF WORDS MODEL
"""
from keras.preprocessing.text import Tokenizer
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(traindf['processed_review'])
list_tokenized_train = tokenizer.texts_to_sequences(traindf['processed_review'])
maxlen = 200
X_pad = pad_sequences(list_tokenized_train, maxlen=maxlen)
"""


"\nfrom keras.preprocessing.text import Tokenizer\nmax_features = 6000\ntokenizer = Tokenizer(num_words=max_features)\ntokenizer.fit_on_texts(traindf['processed_review'])\nlist_tokenized_train = tokenizer.texts_to_sequences(traindf['processed_review'])\nmaxlen = 200\nX_pad = pad_sequences(list_tokenized_train, maxlen=maxlen)\n"

In [5]:
#Word2Vec MODEL
from gensim.models import Word2Vec
embedding_vector_size = 300
wordvec_model = Word2Vec(
    sentences = traindf['processed_review'],
    size = embedding_vector_size,
    min_count=3, window=10, workers=4)

In [6]:
print("Vocabulary size:", len(wordvec_model.wv.vocab))
print("Number of reviews: ",len(traindf))
wordvec_model.wv.most_similar("great")

Vocabulary size: 36749
Number of reviews:  25000


[('excellent', 0.7485159039497375),
 ('wonderful', 0.7191956043243408),
 ('fantastic', 0.7094447016716003),
 ('amaze', 0.6841191649436951),
 ('fine', 0.6785985231399536),
 ('incredible', 0.6731829643249512),
 ('awesome', 0.6718109846115112),
 ('perfect', 0.6689537763595581),
 ('good', 0.648886501789093),
 ('nice', 0.5896484851837158)]

In [7]:
# Code grabbed from https://www.kaggle.com/alexcherniuk/imdb-review-word2vec-bilstm-99-acc
from keras.preprocessing.sequence import pad_sequences
def vectorize_data(data, vocab: dict) -> list:
    print('Vectorize sentences...', end='\r')
    keys = list(vocab.keys())
    filter_unknown = lambda word: vocab.get(word, None) is not None
    encode = lambda review: list(map(keys.index, filter(filter_unknown, review)))
    vectorized = list(map(encode, data))
    print('Vectorize sentences... (done)')
    return vectorized

embed_size = 150
X_pad = pad_sequences(
    sequences=vectorize_data(traindf['processed_review'], vocab=wordvec_model.wv.vocab),
    maxlen=embed_size,
    padding='post')

Using TensorFlow backend.


Vectorize sentences... (done)


In [8]:
y = traindf['sentiment']

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten,CuDNNLSTM
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

embed_size = 150
model = Sequential()
model.add(Embedding(input_dim=wordvec_model.wv.vectors.shape[0],output_dim=wordvec_model.wv.vectors.shape[1],weights=[wordvec_model.wv.vectors], input_length=embed_size))
model.add(Bidirectional(CuDNNLSTM(64)))
#model.add(GlobalMaxPool1D())
model.add(Dropout(0.1))
#model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.summary()



W0930 06:33:48.302227  7408 deprecation_wrapper.py:119] From C:\Users\Ellysah\Anaconda3\envs\keras-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0930 06:33:48.319187  7408 deprecation_wrapper.py:119] From C:\Users\Ellysah\Anaconda3\envs\keras-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0930 06:33:48.323176  7408 deprecation_wrapper.py:119] From C:\Users\Ellysah\Anaconda3\envs\keras-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0930 06:33:48.337136  7408 deprecation_wrapper.py:119] From C:\Users\Ellysah\Anaconda3\envs\keras-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_sessio

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 300)          11024700  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               187392    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 11,212,221
Trainable params: 11,212,221
Non-trainable params: 0
_________________________________________________________________


In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42,shuffle=True)
X_test

array([[  383,   653,   138, ...,     0,     0,     0],
       [   31,  1373, 14058, ...,     0,     0,     0],
       [ 1256,  2775,   327, ...,     0,     0,     0],
       ...,
       [  327,  1261, 20618, ...,  6392,  3574, 22032],
       [ 2031,  4339,  1069, ...,  5551, 16162,  3764],
       [ 1126,   120,    67, ...,     0,     0,     0]])

In [14]:
batch_size = 64
epochs = 3
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 16000 samples, validate on 4000 samples
Epoch 1/3
16000/16000 [==============================] - 24s 1ms/step - loss: 0.1205 - acc: 0.9624 - val_loss: 0.1431 - val_acc: 0.9557
Epoch 2/3
16000/16000 [==============================] - 23s 1ms/step - loss: 0.0510 - acc: 0.9858 - val_loss: 0.1736 - val_acc: 0.9453
Epoch 3/3
16000/16000 [==============================] - 23s 1ms/step - loss: 0.0253 - acc: 0.9936 - val_loss: 0.1904 - val_acc: 0.9447


In [28]:
# validate the model on test dataset to determine generalization
loss, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))

5000/5000 [==============================] - 2s 335us/step

Test accuracy: 95.1%


In [13]:
testdf=pd.read_csv("testData.tsv",header=0, delimiter="\t", quoting=3)
testdf.head(3)


,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
